# TP : eQTL et Randomisation Mendélienne
## Master ST4Health — Génétique Humaine

<div style="background-color:#e8f4f8; padding:12px; border-radius:6px;">

**Objectifs du TP :**
1. Comprendre le concept d'eQTL et son rôle de variable instrumentale
2. Explorer des données de summary statistics GWAS
3. Réaliser une analyse de Randomisation Mendélienne (MR) manuellement
4. Utiliser le package `MendelianRandomization` et interpréter les analyses de sensibilité

**Durée : ~1h40** (après l'introduction magistrale)

</div>

---

> **Note sur les données** : Les données de ce TP sont synthétiques mais réalistes.  
> Les données eQTL sont simulées à partir des valeurs publiées dans GTEx v8 (tissu foie, gène *HMGCR*).  
> Les IVs MR sont basés sur de vrais rsIDs et des tailles d'effet tirées de la littérature  
> (Willer et al. *Nat Genet* 2013 pour le LDL ; Nikpay et al. *Nat Genet* 2015 pour la CAD).  
> Le code pour reproduire les données depuis les sources originales est fourni en annexe.

## 0. Setup

Chargement de `rpy2` et installation des packages R si nécessaire.

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
# Installation des packages si nécessaire (à ne lancer qu'une fois)
if (!require("MendelianRandomization", quietly=TRUE)) {
    install.packages("MendelianRandomization", repos="https://cloud.r-project.org")
}
if (!require("ggplot2", quietly=TRUE)) install.packages("ggplot2", repos="https://cloud.r-project.org")
if (!require("dplyr",   quietly=TRUE)) install.packages("dplyr",   repos="https://cloud.r-project.org")
cat("Packages OK\n")

---
## Partie I — eQTL comme variable instrumentale (~20 min)

### Contexte biologique

La **randomisation mendélienne** repose sur l'idée que les variants génétiques sont aléatoirement distribués dans la population (comme dans un essai randomisé). Pour qu'un SNP soit un bon **instrument génétique**, il doit satisfaire trois hypothèses :

1. **Relevance** : le SNP est associé à l'exposition (ici : il affecte l'expression d'un gène)
2. **Indépendance** : le SNP est indépendant des facteurs confondants
3. **Exclusion restriction** : le SNP n'affecte l'outcome que *via* l'exposition

Un **eQTL** (*expression Quantitative Trait Locus*) est exactement ça : un SNP dont on sait qu'il régule l'expression d'un gène. Il satisfait naturellement l'hypothèse 1.

### Notre exemple : *HMGCR* et le cholestérol LDL

Le gène *HMGCR* code pour la HMG-CoA réductase, enzyme clé de la voie de biosynthèse du cholestérol — et **cible des statines**. Un cis-eQTL dans le foie sur *HMGCR* peut donc servir d'instrument pour estimer l'effet causal du LDL sur les maladies cardiovasculaires.

Le SNP **rs12916** (chr5:74,656,528, build GRCh37) est le lead cis-eQTL de *HMGCR* dans le foie (GTEx v8).

In [ ]:
%%R
# Chargement des données eQTL
eqtl <- read.csv("eqtl_HMGCR.csv")
cat("Dimensions :", nrow(eqtl), "individus x", ncol(eqtl), "variables\n")
head(eqtl)

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 1a** — Décrivez la distribution des génotypes pour rs12916.  
Combien d'individus ont 0, 1 ou 2 copies de l'allèle T ?  
Comparez à la fréquence allélique attendue sous Hardy-Weinberg (MAF = 0.42).

</div>

In [ ]:
%%R
# Votre réponse ici
table(eqtl$genotype_rs12916)

# Fréquences observées vs attendues
n <- nrow(eqtl)
maf <- 0.42
cat("\nFréquences HWE attendues :\n")
cat("  0 copies :", round((1-maf)^2 * n), "\n")
cat("  1 copie  :", round(2*maf*(1-maf) * n), "\n")
cat("  2 copies :", round(maf^2 * n), "\n")

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 1b** — Visualisez la relation entre le génotype rs12916 et l'expression de *HMGCR*.  
Quel est le sens de l'effet ? Est-ce cohérent avec la biologie ?

</div>

In [ ]:
%%R
library(ggplot2)

# Boxplot expression ~ génotype
eqtl$genotype_factor <- factor(eqtl$genotype_rs12916,
                                levels=0:2,
                                labels=c("CC (0T)", "CT (1T)", "TT (2T)"))

ggplot(eqtl, aes(x=genotype_factor, y=HMGCR_expression, fill=genotype_factor)) +
    geom_boxplot(alpha=0.7, outlier.shape=16) +
    geom_jitter(width=0.1, alpha=0.3, size=0.8) +
    scale_fill_manual(values=c("#4575b4","#91bfdb","#e0f3f8")) +
    labs(
        title="cis-eQTL de HMGCR dans le foie",
        subtitle="SNP rs12916 (chr5:74,656,528)",
        x="Génotype rs12916",
        y="Expression HMGCR (log2-normalized)",
        fill="Génotype"
    ) +
    theme_minimal(base_size=13) +
    theme(legend.position="none")

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 1c** — Testez statistiquement l'association eQTL par régression linéaire.  
Ajustez sur l'âge, le sexe et PC1.  
Quelle est la taille d'effet (beta) ? Est-ce significatif ?

</div>

In [ ]:
%%R
# Modèle 1 : eQTL brut (sans covariables)
mod_brut <- lm(HMGCR_expression ~ genotype_rs12916, data=eqtl)
cat("=== Modèle brut ===\n")
summary(mod_brut)$coefficients

# Modèle 2 : ajusté sur covariables
mod_adj <- lm(HMGCR_expression ~ genotype_rs12916 + age + sex + PC1, data=eqtl)
cat("\n=== Modèle ajusté (age, sex, PC1) ===\n")
summary(mod_adj)$coefficients

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 1d** — Calculez la **statistique F** de l'instrument :

$$F = \frac{\hat{\beta}^2}{SE^2}$$

Une règle empirique : F > 10 indique un instrument fort. Qu'en est-il ici ?  
Interprétez : qu'est-ce qu'un instrument *faible* biaiserait dans l'analyse MR ?

</div>

In [ ]:
%%R
beta_eqtl <- coef(summary(mod_adj))["genotype_rs12916", "Estimate"]
se_eqtl   <- coef(summary(mod_adj))["genotype_rs12916", "Std. Error"]

F_stat <- (beta_eqtl / se_eqtl)^2
cat(sprintf("Beta eQTL : %.4f\n", beta_eqtl))
cat(sprintf("SE        : %.4f\n", se_eqtl))
cat(sprintf("F-stat    : %.1f\n", F_stat))
cat(sprintf("\n=> Instrument %s (F %s 10)\n",
    ifelse(F_stat > 10, "FORT", "faible"),
    ifelse(F_stat > 10, ">", "<")))

---
## Partie II — Exploration des données MR (~15 min)

### Passage à l'échelle : du gène au GWAS

L'approche eQTL sur un gène est illustrative, mais une MR robuste utilise **de nombreux instruments génétiques** tirés d'un GWAS sur l'exposition.

Ici l'exposition est le **cholestérol LDL** et l'outcome est la **maladie coronarienne (CAD)**.

Le fichier `MR_IVs_LDL_CAD.csv` contient 77 SNPs instrumentaux déjà sélectionnés et harmonisés :
- Associés au LDL à *p* < 5×10⁻⁸ (seuil GWAS)
- Indépendants (clumpés, r² < 0.01, fenêtre 10 Mb)
- Effets alignés sur l'allèle *augmentant* le LDL
- Présents dans les deux GWAS (LDL et CAD)

In [ ]:
%%R
# Chargement des données MR
mr_data <- read.csv("MR_IVs_LDL_CAD.csv")
cat("Dimensions :", nrow(mr_data), "SNPs x", ncol(mr_data), "variables\n\n")
head(mr_data[, c("SNP","GENE","BETA_LDL","SE_LDL","P_LDL","BETA_CAD","SE_CAD","F_stat")])

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 2a** — Quelle est la distribution des statistiques F dans ce jeu de données ?  
Tous les instruments sont-ils forts ?  
Notez les gènes connus (HMGCR, PCSK9, LDLR, APOE) : sont-ils présents ?

</div>

In [ ]:
%%R
cat("F-stat : min =", round(min(mr_data$F_stat),1),
    " médiane =", round(median(mr_data$F_stat),1),
    " max =", round(max(mr_data$F_stat),1), "\n")
cat("Proportion F > 10 :", mean(mr_data$F_stat > 10), "\n\n")

# Gènes biologiquement importants
genes_cles <- c("HMGCR","PCSK9","LDLR","APOE","APOB","CETP","LPA")
mr_data[mr_data$GENE %in% genes_cles,
        c("SNP","GENE","BETA_LDL","BETA_CAD","F_stat")]

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 2b** — Faites un scatter plot beta_CAD ~ beta_LDL.  
Que remarquez-vous ? Quelle relation attendez-vous si LDL *cause* la CAD ?

</div>

In [ ]:
%%R
ggplot(mr_data, aes(x=BETA_LDL, y=BETA_CAD, label=ifelse(F_stat > 500, GENE, ""))) +
    geom_point(aes(size=F_stat), alpha=0.6, color="#2166ac") +
    geom_smooth(method="lm", formula=y~x-1, se=TRUE,
                color="#d73027", linetype="dashed", linewidth=1) +
    geom_hline(yintercept=0, linetype="dotted", color="grey50") +
    geom_vline(xintercept=0, linetype="dotted", color="grey50") +
    geom_text(hjust=-0.15, vjust=0.5, size=3, color="#555555") +
    labs(
        title="Effets SNP sur LDL-C vs effets SNP sur CAD",
        subtitle="Pente = estimé MR (IVW) — chaque point est un instrument génétique",
        x=expression(hat(beta)[LDL]~"(effet sur le LDL-C, log scale)"),
        y=expression(hat(beta)[CAD]~"(effet sur la CAD, log OR)"),
        size="F-stat"
    ) +
    theme_minimal(base_size=13)

---
## Partie III — MR à la main : IVW (~25 min)

### Principe : du Wald ratio à l'IVW

Pour chaque SNP *j*, le **ratio de Wald** estime l'effet causal de l'exposition sur l'outcome :

$$\hat{\theta}_j = \frac{\hat{\beta}_{Y,j}}{\hat{\beta}_{X,j}}$$

Son erreur standard approximée est :

$$SE(\hat{\theta}_j) \approx \frac{SE_{Y,j}}{|\hat{\beta}_{X,j}|}$$

La méthode **Inverse Variance Weighted (IVW)** combine ces estimés en une moyenne pondérée par l'inverse de la variance :

$$\hat{\theta}_{IVW} = \frac{\sum_j w_j \hat{\theta}_j}{\sum_j w_j} \quad \text{avec} \quad w_j = \frac{1}{SE(\hat{\theta}_j)^2}$$

**Équivalence clé :** l'IVW est strictement équivalent à une régression linéaire pondérée de $\hat{\beta}_Y$ sur $\hat{\beta}_X$ sans intercept, avec poids $1/SE_Y^2$.

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 3a** — Calculez le ratio de Wald et son SE pour chaque SNP.  
Quel SNP a le ratio le plus élevé ? le plus faible ?

</div>

In [ ]:
%%R
# Ratio de Wald par SNP
mr_data$wald_ratio <- mr_data$BETA_CAD / mr_data$BETA_LDL
mr_data$wald_se    <- mr_data$SE_CAD   / abs(mr_data$BETA_LDL)

cat("Wald ratio le plus élevé :\n")
print(mr_data[which.max(mr_data$wald_ratio), c("SNP","GENE","wald_ratio","wald_se")])
cat("\nWald ratio le plus faible :\n")
print(mr_data[which.min(mr_data$wald_ratio), c("SNP","GENE","wald_ratio","wald_se")])

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 3b** — Construisez un **forest plot** des ratios de Wald.  
Que révèle l'hétérogénéité entre les instruments ?

</div>

In [ ]:
%%R
library(dplyr)

# Ordonner par wald_ratio pour le forest plot
fd <- mr_data %>%
    arrange(wald_ratio) %>%
    mutate(
        ci_lo = wald_ratio - 1.96 * wald_se,
        ci_hi = wald_ratio + 1.96 * wald_se,
        SNP_label = paste0(SNP, " (", GENE, ")")
    )
fd$SNP_label <- factor(fd$SNP_label, levels=fd$SNP_label)

ggplot(fd, aes(x=wald_ratio, y=SNP_label)) +
    geom_point(color="#2166ac", size=1.5) +
    geom_errorbarh(aes(xmin=ci_lo, xmax=ci_hi), height=0, color="#2166ac", alpha=0.5) +
    geom_vline(xintercept=0, linetype="dashed", color="grey40") +
    labs(
        title="Forest plot — Ratios de Wald par instrument",
        x="Wald ratio (effet causal estimé LDL → CAD)",
        y=NULL
    ) +
    theme_minimal(base_size=9) +
    theme(axis.text.y=element_text(size=7))

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 3c** — Calculez l'estimé IVW **manuellement** (moyenne pondérée des ratios de Wald).  
Calculez le SE, l'IC 95% et la p-value.  
Interprétez : est-ce que le LDL cause la CAD ?

</div>

In [ ]:
%%R
# IVW = moyenne pondérée des ratios de Wald
weights     <- 1 / mr_data$wald_se^2
ivw_est     <- sum(weights * mr_data$wald_ratio) / sum(weights)
ivw_se      <- sqrt(1 / sum(weights))
ivw_z       <- ivw_est / ivw_se
ivw_pval    <- 2 * (1 - pnorm(abs(ivw_z)))
ivw_ci_lo   <- ivw_est - 1.96 * ivw_se
ivw_ci_hi   <- ivw_est + 1.96 * ivw_se

cat("=== IVW (moyenne pondérée) ===\n")
cat(sprintf("Estimé  : %.4f\n", ivw_est))
cat(sprintf("SE      : %.4f\n", ivw_se))
cat(sprintf("IC 95%%  : [%.4f ; %.4f]\n", ivw_ci_lo, ivw_ci_hi))
cat(sprintf("p-value : %.2e\n", ivw_pval))
cat(sprintf("\n=> Pour 1 unité d'augmentation du LDL (log scale),\n"))
cat(sprintf("   l'odds ratio de CAD est : %.3f (IC: %.3f - %.3f)\n",
    exp(ivw_est), exp(ivw_ci_lo), exp(ivw_ci_hi)))

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 3d** — Vérifiez votre résultat par **régression WLS sans intercept**.  
Les deux méthodes donnent-elles exactement le même résultat ? Pourquoi ?

</div>

In [ ]:
%%R
# IVW = régression WLS de beta_CAD ~ beta_LDL, sans intercept, poids = 1/SE_CAD^2
mod_ivw <- lm(BETA_CAD ~ -1 + BETA_LDL,
              weights = 1/SE_CAD^2,
              data = mr_data)

ivw_reg_est <- coef(mod_ivw)["BETA_LDL"]
ivw_reg_se  <- sqrt(vcov(mod_ivw)["BETA_LDL","BETA_LDL"])

cat("=== IVW (régression WLS) ===\n")
cat(sprintf("Estimé  : %.4f\n", ivw_reg_est))
cat(sprintf("SE      : %.4f\n", ivw_reg_se))
cat(sprintf("\nDifférence avec méthode précédente : %.6f\n",
    abs(ivw_est - ivw_reg_est)))
cat("(Les deux méthodes sont strictement équivalentes)\n")

---
## Part IV — `MendelianRandomization` package and sensitivity analyses (~30 min)

### Why sensitivity analyses?

IVW assumes **all instruments are valid** (no direct pleiotropy).  
In practice, some SNPs may affect the outcome through pathways independent of the exposure: **horizontal pleiotropy**.

Three complementary methods test the robustness of the IVW estimate:

| Method | Key assumption | Robust when |
|---|---|---|
| **IVW** | All IVs valid | No pleiotropy |
| **MR-Egger** | Pleiotropy proportional to effect (InSIDE) | Directional pleiotropy |
| **Weighted Median** | ≥ 50% of IV weight from valid IVs | Majority valid |
| **Weighted Mode** | Most IVs share the same causal estimate | Cluster of valid IVs |

### IV.1 — Create the MR input object

In [ ]:
%%R
library(MendelianRandomization)

# Create MR input object from harmonised data
MR_input <- mr_input(
    bx       = mr_data$BETA_LDL,
    bxse     = mr_data$SE_LDL,
    by       = mr_data$BETA_CAD,
    byse     = mr_data$SE_CAD,
    snps     = mr_data$SNP,
    exposure = "LDL-cholesterol",
    outcome  = "Coronary artery disease"
)
cat("MR input object created\n")
cat("Class    :", class(MR_input), "\n")
cat("N instruments:", length(MR_input@betaX), "\n")

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 4a** — Run `mr_ivw()` and compare with your manual IVW from Part III.  
Are the estimates identical? Check the heterogeneity statistic (Cochran's Q).  
What does significant Q tell you about the instruments?

</div>

In [ ]:
%%R
# IVW with the package
res_ivw <- mr_ivw(MR_input)
res_ivw

# Compare with manual estimate from Part III
weights  <- 1 / mr_data$wald_se^2
ivw_manual <- sum(weights * mr_data$wald_ratio) / sum(weights)
cat(sprintf("\nManual IVW  : %.6f\n", ivw_manual))
cat(sprintf("Package IVW : %.6f\n", res_ivw@Estimate))
cat(sprintf("Difference  : %.2e\n", abs(ivw_manual - res_ivw@Estimate)))

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 4b** — Run all methods with `mr_allmethods()`.  
Are the estimates consistent across methods?  
Which method gives the widest confidence interval, and why?

</div>

In [ ]:
%%R
# All main methods in one call
res_all <- mr_allmethods(MR_input, method = "main")
res_all

In [ ]:
%%R
# Visual comparison
mr_plot(res_all)

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 4c** — MR-Egger: interpret the **intercept**.  
- What does a non-zero intercept indicate biologically?  
- Is the InSIDE assumption testable?  
- Is there evidence of directional pleiotropy here?

</div>

In [ ]:
%%R
res_egger <- mr_egger(MR_input)

cat("=== MR-Egger ===\n")
cat(sprintf("Slope (causal effect) : %.4f  SE: %.4f  p: %.2e\n",
    res_egger@Estimate,   res_egger@StdError.Est, res_egger@Pvalue.Est))
cat(sprintf("Intercept (pleiotropy): %.4f  SE: %.4f  p: %.3f\n",
    res_egger@Intercept,  res_egger@StdError.Int, res_egger@Pvalue.Int))
cat(sprintf("\n=> Intercept %s significantly different from 0 (p %s 0.05)\n",
    ifelse(res_egger@Pvalue.Int < 0.05, "IS", "is NOT"),
    ifelse(res_egger@Pvalue.Int < 0.05, "<", ">=")))

### IV.2 — Scatter plot with all regression lines

In [ ]:
%%R
# Scatter plot with IVW, MR-Egger and Weighted Median lines
mr_plot(MR_input,
        line     = "ivw",
        orientate = TRUE,
        interactive = FALSE) +
    ggplot2::labs(
        title    = "MR LDL-C → CAD — Scatter plot",
        subtitle = "Each point = one genetic instrument"
    )

### IV.3 — Forest plot of individual Wald ratios

In [ ]:
%%R
# Forest plot — one point per instrument
mr_forest(MR_input, ordered = TRUE)

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 4d — Bonus: MR-PRESSO** (outlier detection)  
MR-PRESSO detects instruments with outlying Wald ratios that may be pleiotropic.  
It corrects the IVW estimate after removing outliers.

Run it and interpret:
- Is the global test significant (evidence of pleiotropy)?  
- Which SNPs are flagged as outliers?  
- Does the corrected estimate differ from the raw IVW?

</div>

In [ ]:
%%R
# MR-PRESSO (can take 1-2 min with n_iterations=5000)
# Requires: install.packages("MRPRESSO")
# library(MRPRESSO)
# res_presso <- mr_presso(
#     BetaOutcome    = "BETA_CAD",
#     BetaExposure   = "BETA_LDL",
#     SdOutcome      = "SE_CAD",
#     SdExposure     = "SE_LDL",
#     OUTLIERtest    = TRUE,
#     DISTORTIONtest = TRUE,
#     data           = mr_data,
#     NbDistribution = 5000
# )
# res_presso$`Main MR results`
# res_presso$`MR-PRESSO results`$`Global Test`

cat("MR-PRESSO commenté pour gagner du temps.\n")
cat("Décommenter et relancer si vous souhaitez tester.\n")

<div style="background-color:#f5f1f6; padding:10px; border-radius:4px;">

**Question 4e — Bonus: Reverse MR** (bidirectionality check)  
Does CAD cause LDL? Swap exposure and outcome and re-run IVW.  
What do you conclude about causal directionality?

</div>

In [ ]:
%%R
# Reverse MR: CAD -> LDL
MR_input_rev <- mr_input(
    bx       = mr_data$BETA_CAD,
    bxse     = mr_data$SE_CAD,
    by       = mr_data$BETA_LDL,
    byse     = mr_data$SE_LDL,
    snps     = mr_data$SNP,
    exposure = "Coronary artery disease",
    outcome  = "LDL-cholesterol"
)
res_ivw_rev <- mr_ivw(MR_input_rev)
cat("=== Reverse MR: CAD -> LDL ===\n")
res_ivw_rev

---
## Summary

<div style="background-color:#e8f4f8; padding:12px; border-radius:6px;">

**Synthesis questions:**

1. Summarise in 3 sentences the causal evidence between LDL and CAD from this practical.

2. Are IVW, MR-Egger and Weighted Median consistent?  
   What does this imply about instrument validity?

3. Is the MR-Egger intercept concerning here? Under what circumstances would it be?

4. Why is the *HMGCR* cis-eQTL a particularly interesting instrument  
   for testing the effect of statins by MR?

5. **Bonus**: the SNP in the *LPA* gene has a Wald ratio that differs from the others.  
   Can you find a biological explanation — is this pleiotropy or a genuine biological effect?

</div>

---

## Going further (optional)

- **Multivariable MR (MVMR)**: estimate the effect of multiple exposures simultaneously  
  (e.g. LDL + HDL + triglycerides → CAD). Available in `MendelianRandomization`.
- **Colocalization**: test whether the eQTL signal and the GWAS signal at the same locus  
  are the same causal signal (`coloc` package).
- **Steiger filtering**: formally test the direction of causality for each instrument.

---

*eQTL data simulated from GTEx v8 (Liver, HMGCR, rs12916 — GTEx Consortium 2020).*  
*MR data based on Willer et al. Nat Genet 2013 (LDL) and Nikpay et al. Nat Genet 2015 (CAD).*  
*MR-Egger: Bowden et al. Int J Epidemiol 2015. Weighted Median: Bowden et al. Genet Epidemiol 2016.*